In [ ]:
!pip install herbie-data --quiet

In [ ]:
# linting
%load_ext nb_black
%load_ext lab_black

# Skill from ECMWF downloaded with `herbie`

[`herbie`](https://herbie.readthedocs.io/en/latest/user_guide/_tutorial_notebooks/fast.html) downloads forecasts data easily. The resulting datasets is out-of-the-box compatible with `climpred`.

In [ ]:
import xarray as xr
import numpy as np

import climpred  # forecast verification

In [ ]:
from herbie import Herbie

H = Herbie(date="2022-01-27 00:00", model="ecmwf", product="enfo", fxx=24 * 1)
ds = H.xarray(":2t:")
ds

In [ ]:
# take the first with multiple members as forecast
init = ds[0][["t2m"]]

In [ ]:
H = Herbie(date="2022-01-28 00:00", model="ecmwf", product="enfo", fxx=0)
ds = H.xarray(":2t:")
ds

In [ ]:
# take first and make ensemble member mean as observations
obs = ds[0].mean("number").drop(["step", "valid_time"]).expand_dims("time")[["t2m"]]

## Forecast skill verification

Using using {py:class}`.HindcastEnsemble`.

`climpred` expects `init`, `lead` and optional `member` as dimensions, see [setting-up-your-dataset](setting-up-data.html#setting-up-your-dataset). Existing dimensions are renamed automatically if CF `standard_names` match.

In [ ]:
hindcast = climpred.HindcastEnsemble(
    init.expand_dims(["time", "step"])
).add_observations(obs)

hindcast

In [ ]:
skill = hindcast.verify(
    metric="crps", comparison="m2o", dim=["init", "member"], alignment="same_init"
)

In [ ]:
skill.t2m.plot(robust=True)